In [1]:
import psycopg2
# Connect to an existing database
conn = psycopg2.connect("dbname='bt' user='postgres' host='localhost' password='severo' port='5432'")

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a command: this creates a new table
cur.execute("select m.tid,m.time,m.edge_id from matched_tracks m where m.tid<=1 order by m.tid,m.time;")
matched_tracks = cur.fetchall()
cur.execute("select distinct edge_id from matched_tracks where tid<=1 order by edge_id asc")
egde_ids = cur.fetchall()
cur.execute("select i.tid,i.start_time,i.end_time,EXTRACT(EPOCH FROM (i.end_time::timestamp - i.start_time::timestamp)) from ib_stops_carlos i where i.tid<=1 order by i.tid, start_time;")
stops = cur.fetchall()
# Pass data to fill a query placeholders and let Psycopg perform
# the correct conversion (no more SQL injections!)
#>>> cur.execute("INSERT INTO test (num, data) VALUES (%s, %s)",
#...      (100, "abc'def"))

# Query the database and obtain data as Python objects
#>>> cur.execute("SELECT * FROM test;")
#>>> cur.fetchone()
#(1, 100, "abc'def")

# Make the changes to the database persistent
#>>> conn.commit()

# Close communication with the database
cur.close()

conn.close()

In [2]:
# função que verificar se um ponto é um stop.
def verifica(ponto,stops):
    for stop in stops:
        if ponto[0]==stop[0] and ponto[1]==stop[1]:
            return stop[3]
    return -1     

In [3]:
# Encrontando as trajetorias.
trajetorias = list()
trajetoria = list()
taxi_id = 1

for ponto in matched_tracks:
    if ponto[0]==taxi_id:
        diferenca=verifica([ponto[0],ponto[1]],stops)
        if diferenca==-1:
            trajetoria.append(ponto)
        else:
            trajetoria.append(ponto)
            if diferenca > 120:
                trajetorias.append(trajetoria)
                trajetoria = list()
    else:
        trajetorias.append(trajetoria)
        trajetoria = list()
        trajetoria.append(ponto)
        taxi_id=taxi_id+1     

In [4]:
# Criando dicionário para mapeamento dos egde_ids para os indices da matriz m:
dicionario1=dict()
indice=0
for egde_id in egde_ids:
    dicionario1[egde_id[0]]=indice
    indice=indice+1
# Criando dicionário para o mapeamento dos indices da matriz para os egdes_ids:
dicionario2 = dict()
for i in egde_ids:
    d=dicionario1[i[0]]
    dicionario2[d]=i[0]

In [30]:
# Criando a matriz de transições:
import numpy as np
k=len(dicionario1)
m =np.zeros( (k,k),dtype=np.int32)

In [31]:
# Contando quantas transições exites de cada egde_id para outro:
for traj in trajetorias:
    c = len(traj)
    for i in range(0,c-1):
        if traj[i][2] != traj[i+1][2]:
            d1=dicionario1[traj[i][2]]
            d2=dicionario1[traj[i+1][2]]
            m[d1][d2]=m[d1][d2]+1

In [7]:
def not_marked(m,a):
    for n in m:
        for i in n:
            if i > 0 and i>=a:
                return True
    return False

def maximo(m,c):
    l=0
    f=0
    for i in range(0,c):
        for j in range(0,c):
            if m[i][j]>m[l][f]:
                l=i
                f=j
    return [l,f]

def marking(a):
    if a>0:
        return -1*a
    return a

def not_marked_2(m,d,f,a,e,c):
    u=-1
    minimo=100000
    for i in range(0,c):
        if m[f][i]>=a:
            if abs(m[d][f]-m[f][i]) < minimo and abs(m[d][f]-m[f][i])<=e:
                minimo = abs(m[d][f]-m[f][i])
                u=i
    return u

def not_marked_3(m,d,f,a,e,c):
    u=-1
    minimo=1000000
    for i in range(0,c):
        if m[i][d]>=a:
            if abs(m[d][f]-m[u][d])<minimo and abs(m[d][f]-m[u][d])<=e:
                minimo=abs(m[d][f]-m[u][d])
                u=i
    return u

In [23]:
def netscan_phase_1(dicionario2,m,e,a):
    O=set()
    k=0
    C=list()
    while not_marked(m,a):
        indices=maximo(m, len(dicionario2))
        d=indices[0]
        f=indices[1]
        Sd=dicionario2[d]
        Sf=dicionario2[f]
        C.append([Sd,Sf])
        m[d][f]=marking(m[d][f])
        u=not_marked_2(m,d,f,a,e,len(dicionario2))
        while u>-1:
            Su=dicionario2[u]
            C[k].append(u)
            m[f][u]=marking(m[f][u])
            d=f
            f=u
            u=not_marked_2(m,d,f,a,e,len(dicionario2))
        u=not_marked_3(m,d,f,a,e,len(dicionario2))
        while u>-1:
            Su=dicionario2[u]
            C[k].append(Su)
            m[u][d]=marking(m[u][d])
            f=d
            d=u
            u=not_marked_3(m,d,f,a,e,len(dicionario2))
        k=k+1
    return C

In [32]:
C=netscan_phase_1(dicionario2,m,10,1)

In [33]:
for c in C:
    print(c)

[25649, 5299, 39, 218, 80, 218, 81, 218, 219, 216, 217, 3, 4, 45, 74, 77, 76, 194, 68, 63, 64, 65, 66, 67, 144, 145, 146, 22, 23, 24, 42, 135, 136, 137, 138, 139, 140, 141, 142, 143, 85, 86, 41, 148, 149, 150, 83, 3, 82, 3, 83, 151, 118, 62, 25, 26, 158, 168, 159, 0, 1, 2, 120, 160, 161, 162, 163, 198, 157, 60, 61, 58, 62, 119, 195, 196, 164, 197, 152, 93, 94, 95, 5, 153, 154, 62, 129, 133, 132, 130, 131, 34, 35, 36, 134, 177, 43, 44, 176, 147, 235, 58, 235, 59, 235, 75, 69, 70, 71, 72, 73, 78, 236, 237, 10, 11, 37, 184, 191, 193, 187, 155, 178, 179, 188, 182, 200, 183, 185, 180, 181, 156, 192, 186, 189, 190, 20, 21, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 221, 222, 223, 225, 226, 227, 228, 238, 232, 30, 230, 31, 32, 230, 231, 33, 224, 6, 12, 13, 14, 229, 14, 3406, 3405, 64140]
[5299, 25649]
[14029, 38066, 49, 50, 91, 92, 51, 96, 52, 53, 54, 55, 98, 56, 98, 57, 98, 99, 84]
[38074, 14029, 115, 29802, 9494, 25370, 38076, 38075, 14029, 38065, 38064, 3806

In [34]:
print(m)

[[ 0 -1  0 ...  0  0  0]
 [ 0  0 -1 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0 -1  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


In [13]:
import math
raiz=math.ceil(math.sqrt(count))
print(raiz)

159


In [15]:
n =np.zeros( (raiz,raiz),dtype=np.int32)

In [19]:
dicionario3=dict()
l=0
c=0
for i in range(0,k):
    for j in range(0,k):
        if m[i][j] !=0:
            n[l][c]=m[i][j]
            dicionario3[(l,c)]=(i,j)
            c=c+1
            if c==raiz:
                c=0
                l=l+1

In [22]:
maxi=n[0][0]
for t in n:
    for i in t:
        if i>maxi:
            maxi= i
print(maxi)

30
